In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
# default_exp quick

# Quick

This module provides some functions with a default toolbox for parameter tuning.

In [ ]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script

## Quick eaSimple interface

In [ ]:
#export
from eptune.algorithms import eaSimpleWithExtraLog
from eptune.utils import ConcurrentMap
from eptune.crossover import cxDictUniform
from eptune.mutation import mutDictRand
from deap import base, creator, tools
from functools import partial
import random
import numpy

default_stats = tools.Statistics(lambda ind: ind.fitness.values)
default_stats.register("avg", numpy.mean, axis=0)
default_stats.register("std", numpy.std, axis=0)
default_stats.register("min", numpy.min, axis=0)
default_stats.register("max", numpy.max, axis=0)

def qtuneSimple(params,
                evaluate,
                n_pop=10,
                cxpb=0.6,
                mutpb=0.6,
                ngen=10,
                hof=2,
                elitism=True,
                stats=default_stats,
                crossover=partial(cxDictUniform, indpb=0.6),
                select=partial(tools.selTournament, tournsize=3),
                mutate=partial(mutDictRand, indpb=0.7),
                n_jobs=1,
                seed=None,
                verbose=__debug__):
    """Quick tune using `eaSimpleWithExtraLog`. Just provide parameter list and function to tune.
    The function given should accept keywords arguments in the parameter list.
    Check examples below for more information."""
    if seed is not None:
        random.seed(seed)

    def initParams(cls):
        return cls({i.name: next(i) for i in cls.params})

    if hof > 0:
        hof = tools.HallOfFame(hof)
    else:
        hof = None

    creator.create("eptLoss", base.Fitness, weights=(-1.0, ))
    creator.create("eptParameters",
                   dict,
                   params=params,
                   fitness=creator.eptLoss)
    toolbox = base.Toolbox()
    toolbox.register("individual", initParams, creator.eptParameters)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register('evaluate', evaluate)
    toolbox.register("select", select)
    toolbox.register("mate", crossover)
    toolbox.register("mutate", mutate, params=params)
    with ConcurrentMap(n_jobs) as pmap:
        toolbox.register('map', pmap)
        population, logbook = eaSimpleWithExtraLog(toolbox.population(n_pop),
                                   toolbox,
                                   cxpb=cxpb,
                                   mutpb=mutpb,
                                   ngen=ngen,
                                   halloffame=hof,
                                   elitism=elitism,
                                   stats=stats,
                                   verbose=verbose)
    del creator.eptLoss
    del creator.eptParameters
    return population, logbook, hof

### Example of using `qtuneSimple`

In [ ]:
from eptune.parameter import *
params = [
    FloatParameter([-100, 200], 'x'),
    FloatParameter([-100, 200], 'y')
]


def evaluate(params):
    return [params['x']**2+params['y']**2]


In [ ]:
%%time
popu, logbook, hof = qtuneSimple(params,evaluate,cxpb=0.6,
                            n_pop=30, mutpb=0.6,
                            ngen=500, elitism=False,
                            seed=42, verbose=False)
fig = logbook.plot(['min','avg'])

In [ ]:
hof

In [ ]:
%%time
popu, logbook, hof = qtuneSimple(params,evaluate,cxpb=0.6,
                            n_pop=30, mutpb=0.6,
                            ngen=500, elitism=True,
                            seed=42, verbose=False)
fig = logbook.plot(['min','avg'])

In [ ]:
hof

## Quick iterator interface

In [ ]:
#export
import queue
import threading


class qtuneIterate:
    def __init__(self,
                 params,
                 n_pop=10,
                 cxpb=0.6,
                 mutpb=0.6,
                 ngen=10,
                 hof=2,
                 elitism=True,
                 stats=default_stats,
                 crossover=partial(cxDictUniform, indpb=0.6),
                 select=partial(tools.selTournament, tournsize=3),
                 mutate=partial(mutDictRand, indpb=0.7),
                 seed=None,
                 verbose=__debug__):
        self.parameter = queue.Queue()
        self.population = None

        if seed is not None:
            random.seed(seed)

        def initParams(cls):
            return cls({i.name: next(i) for i in cls.params})

        def evaluate(params):
            def wait_fitness(params, cond):
                self.parameter.put((params, cond))
                with cond:
                    cond.wait()

            cond = threading.Condition()
            eathread = threading.Thread(name='evaluate',
                                        target=wait_fitness,
                                        args=(params, cond))
            eathread.start()
            eathread.join()
            return self._result

        if hof > 0:
            self.hof = tools.HallOfFame(hof)
        else:
            self.hof = None
        if hasattr(creator, 'eptLoss'):
            del creator.eptLoss
        if hasattr(creator, 'eptParameters'):
            del creator.eptParameters
        creator.create("eptLoss", base.Fitness, weights=(-1.0, ))
        creator.create("eptParameters",
                       dict,
                       params=params,
                       fitness=creator.eptLoss)
        self.toolbox = base.Toolbox()
        self.toolbox.register("individual", initParams, creator.eptParameters)
        self.toolbox.register("population", tools.initRepeat, list,
                              self.toolbox.individual)
        self.toolbox.register('evaluate', evaluate)
        self.toolbox.register("select", select)
        self.toolbox.register("mate", crossover)
        self.toolbox.register("mutate", mutate, params=params)
        # Currently this class is not threadsafe, so the sequential map is forced.
        self.toolbox.register('map', map)

        def eaWrapper():
            self.population, self.logbook = eaSimpleWithExtraLog(
                self.toolbox.population(n_pop),
                self.toolbox,
                cxpb=cxpb,
                mutpb=mutpb,
                ngen=ngen,
                halloffame=self.hof,
                elitism=elitism,
                stats=stats,
                verbose=verbose)

        self._target = eaWrapper

    def set_result(self, result, cond):
        self._result = result
        with cond:
            cond.notifyAll()

    def get_ctx(self, timeout=None):
        if self.population is None:
            while timeout is None or timeout > 0:
                try:
                    return self.parameter.get(timeout=1)

                except queue.Empty:
                    if self.population is not None:
                        raise StopIteration('Done')
                if timeout is not None:
                    timout -= 1
        else:
            raise StopIteration('Done')

    def __next__(self):
        return self.get_ctx()

    def __iter__(self):
        return self

    def __call__(self, clear_hof=False):
        self.population = None
        if clear_hof and self.hof is not None:
            self.hof.clear()
        self._eaThread = threading.Thread(name='eaThread', target=self._target)
        self._eaThread.start()
        return self

### Example of using `qtuneIterate`

In [ ]:
from eptune.parameter import *
qi = qtuneIterate(
    [IntegerParameter([1, 100], 'x'),
    FloatParameter([1, 100], 'y')],
    n_pop=20,
    cxpb=0.6,
    mutpb=0.3,
    ngen=12,
    hof=2,
    elitism=True,
    seed=42
)

In [ ]:
for p, c in qi():
    qi.set_result([(p['x']+p['y']-15)**2], c)

In [ ]:
qi.hof

In [ ]:
fig = qi.logbook.plot(['avg','min'])